In [12]:
!pip install -q html2text  lxml  httpx

In [13]:
from html2text import HTML2Text
from json import dumps,loads
from lxml.html.clean import Cleaner
from lxml.html import HtmlElement, tostring, fromstring
import lxml, httpx
import copy
import re

In [14]:
def html_repr(self):
    return lxml.html.tostring(self, encoding='unicode', pretty_print=True)

HtmlElement.__repr__ = html_repr
HtmlElement.__str__ = html_repr

In [15]:
def get_body(url):
    body = fromstring(httpx.get(url).text).xpath('//body')[0] #. for converting to HtmlElement
    assert type(body) == HtmlElement
    body = Cleaner(javascript=True, style=True).clean_html(body)
    return fromstring(''.join(tostring(c, encoding='unicode') for c in body))

In [16]:
body = get_body('https://example.com/')
body

<div>
    <h1>Example Domain</h1>
    <p>This domain is for use in illustrative examples in documents. You may use this
    domain in literature without prior coordination or asking for permission.</p>
    <p><a href="https://www.iana.org/domains/example">More information...</a></p>
</div>


In [17]:
type(body) == lxml.html.HtmlElement

True

### generator for finding list of xpath

In [18]:
from typing import List
import traceback
def find_elements_by_xpaths(body:HtmlElement, xpaths:List[str]):
    for xpath in xpaths:
        try:
            yield body.xpath(xpath)
        except Exception as e:
            traceback.print_exc()
            print(e)
            yield []
xpaths = ['//h1', "//p[1]"]

# Extract elements
results = list(find_elements_by_xpaths(body, xpaths))
results

[[<h1>Example Domain</h1>
      ],
 [<p>This domain is for use in illustrative examples in documents. You may use this
      domain in literature without prior coordination or asking for permission.</p>
      ]]

### delete elements given xpaths using `find_elements_by_xpaths`

In [19]:
def delete_elements_by_xpath(body, xpaths):

    lis_html_eles = find_elements_by_xpaths(body, xpaths)
    #print(lis_html_eles)
    #print(xpaths)
    for i, elements in enumerate(lis_html_eles):
        #print(len(elements))
        print(f"{xpaths[i]}--->{len(elements)}")
        for element in elements:
            try:
              parent = element.getparent()
              #print(parent)
              if parent is not None:
                  #print("element.text_content()")
                  #print(parent.text_content())
                  parent.remove(element)
            except Exception as e:
                traceback.print_exc()
                print(e)

new_body = copy.deepcopy(body)
delete_elements_by_xpath(new_body, xpaths)
new_body

//h1--->1
//p[1]--->1


<div>
    <p><a href="https://www.iana.org/domains/example">More information...</a></p>
</div>


In [20]:
def get_md(cts):
  h2t = HTML2Text(bodywidth=500000)
  h2t.ignore_links = True
  h2t.mark_code = True
  h2t.ignore_images = True
  res = h2t.handle(str(cts))
  return res
  """
  def _f(m): return f'```\n{dedent(m.group(1))}\n```'
  return re.sub(r'\[code]\s*\n(.*?)\n\[/code]', _f, res or '', flags=re.DOTALL).strip()
  """
get_md(new_body)

'More information...\n\n'

### remove elements which are below an element

In [21]:
def get_following_siblings(element: HtmlElement):
    try:
        next_sibling = element.getnext()

        while next_sibling is not None:
            yield next_sibling
            next_sibling = next_sibling.getnext()

    except Exception as e:
        print(f"An error occurred while getting following siblings: {str(e)}")
        return None

h1 = body.xpath('h1')[0]
print(f"{h1=}")
slibling_lis = list(get_following_siblings(h1))
print(f"{slibling_lis=}")

h1=<h1>Example Domain</h1>
    

slibling_lis=[<p>This domain is for use in illustrative examples in documents. You may use this
    domain in literature without prior coordination or asking for permission.</p>
    
, <p><a href="https://www.iana.org/domains/example">More information...</a></p>

]


In [22]:
body

<div>
    <h1>Example Domain</h1>
    <p>This domain is for use in illustrative examples in documents. You may use this
    domain in literature without prior coordination or asking for permission.</p>
    <p><a href="https://www.iana.org/domains/example">More information...</a></p>
</div>


In [23]:
def delete_all_following_elements_by_xpath(root: HtmlElement, xpath: str) -> int:
    """Delete all following sibling elements of the element found by the given XPath."""
    try:
        # Find the element using the provided XPath
        element = root.xpath(xpath)[0]
        print(f"{element.text_content()}")
        removed_count = 0
        following = list(get_following_siblings(element))

        # Get the following siblings using the generator
        for sibling in following:
            if sibling is None:
                continue
            try:
                sibling.getparent().remove(sibling)
                removed_count += 1
                print(f"Removed element: {sibling.text_content()}")
            except Exception as e:
                print(f"Failed to remove element: {sibling}. Error: {str(e)}")
                traceback.print_exc()

        return removed_count

    except Exception as e:
        print(f"An unexpected error occurred: {str(e)}")
        traceback.print_exc()
        return 0
#print(body)
#delete_all_following_elements_by_xpath(body, 'h1')
#print("------------")
#print(body)

## wiki

In [24]:
body1 = get_body('https://or.wikipedia.org/wiki/%E0%AC%AA%E0%AD%8D%E0%AC%B0%E0%AC%A6%E0%AD%8D%E0%AD%9F%E0%AD%81%E0%AC%AE%E0%AD%8D%E0%AC%A8_%E0%AC%B2%E0%AD%87%E0%AC%99%E0%AD%8D%E0%AC%95%E0%AC%BE')
body = body1.xpath('//div[@id="mw-content-text"]')[0]

In [25]:
adhar = body.xpath('//div[h2[@id="ଆଧାର"]]')[0]
adhar.text_content()

'ଆଧାର[ସମ୍ପାଦନା]'

In [26]:
l = list(get_following_siblings(adhar))
for i in range(len(l)):
  print(l[i].text_content())
  print("---------")



↑ "କାହାଣୀରେ ବଡ଼ମାନଙ୍କ ଭୂମିକା ଆଗଭଳି ନାହିଁ". ଧରିତ୍ରୀ. ୧୯ ଫେବୃଆରୀ ୨୦୧୪. p. ୧୧. Retrieved 31 October 2015. {{cite news}}: Check date values in: |date= (help)


---------
ଅଧିକ ତଥ୍ୟ[ସମ୍ପାଦନା]
---------
ଇଣ୍ଟରନେଟ ମୁଭି ଡାଟାବେସରେ ପ୍ରଦ୍ୟୁମ୍ନ ଲେଙ୍କା
---------



ଉଇକିମିଡ଼ିଆ କମନ୍ସରେ  ପ୍ରଦ୍ୟୁମ୍ନ ଲେଙ୍କା
ବାବଦରେ ମାଧ୍ୟମ ରହିଛି ।

---------


In [27]:
adhar_xpath = '//div[h2[@id="ଆଧାର"]]'
delete_all_following_elements_by_xpath(body1, adhar_xpath)
#get_md(body)

ଆଧାର[ସମ୍ପାଦନା]
Removed element: 

↑ "କାହାଣୀରେ ବଡ଼ମାନଙ୍କ ଭୂମିକା ଆଗଭଳି ନାହିଁ". ଧରିତ୍ରୀ. ୧୯ ଫେବୃଆରୀ ୨୦୧୪. p. ୧୧. Retrieved 31 October 2015. {{cite news}}: Check date values in: |date= (help)


Removed element: ଅଧିକ ତଥ୍ୟ[ସମ୍ପାଦନା]
Removed element: ଇଣ୍ଟରନେଟ ମୁଭି ଡାଟାବେସରେ ପ୍ରଦ୍ୟୁମ୍ନ ଲେଙ୍କା
Removed element: 


ଉଇକିମିଡ଼ିଆ କମନ୍ସରେ  ପ୍ରଦ୍ୟୁମ୍ନ ଲେଙ୍କା
ବାବଦରେ ମାଧ୍ୟମ ରହିଛି ।



4

In [28]:
#get_md(body1)

In [29]:
xpaths_to_remove = [
    '//div[@class="navbox"]',
    '//div'
    '//a[@class="mw-jump-link"]',
    '//div[@class="vector-header-container"]',
    '//span[@class="plainlinks"]',
    '//nav',
    '//div[@class="vector-page-toolbar"]',
    '//div[@class="title-shortlink-container"]',
    '//div[@id="p-lang-btn"]',
    '//div[@id="siteSub"]',
    '//div[@class="vector-page-toolbar"]',
    '//table[contains(@class, "infobox")]',
    '//span[@class="mw-editsection"]',
    '//div[h2[@id="ଆଧାର"]]',
    '//div[@class="reflist"]',
    '//div[@class="printfooter"]',
    '//div[h2[@id="ବାହାର_ଆଧାର"]]//following::ul',
    '//div[h2[@id="ବାହାର_ଆଧାର"]]',
    '//div[@id="catlinks"]',
    '//div[@class="mw-footer-container"]',
    '//sup',
]
delete_elements_by_xpath(body1, xpaths_to_remove)

//div[@class="navbox"]--->0
//div//a[@class="mw-jump-link"]--->1
//div[@class="vector-header-container"]--->1
//span[@class="plainlinks"]--->2
//nav--->8
//div[@class="vector-page-toolbar"]--->1
//div[@class="title-shortlink-container"]--->0
//div[@id="p-lang-btn"]--->1
//div[@id="siteSub"]--->1
//div[@class="vector-page-toolbar"]--->0
//table[contains(@class, "infobox")]--->1
//span[@class="mw-editsection"]--->8
//div[h2[@id="ଆଧାର"]]--->1
//div[@class="reflist"]--->0
//div[@class="printfooter"]--->1
//div[h2[@id="ବାହାର_ଆଧାର"]]//following::ul--->0
//div[h2[@id="ବାହାର_ଆଧାର"]]--->0
//div[@id="catlinks"]--->1
//div[@class="mw-footer-container"]--->1
//sup--->1


In [30]:
get_md(body1)

'# ପ୍ରଦ୍ୟୁମ୍ନ ଲେଙ୍କା\n\n****\n\n**ପ୍ରଦ୍ୟୁମ୍ନ ଲେଙ୍କା** ଜଣେ ଓଡ଼ିଆ କଥାଚିତ୍ର ଅଭିନେତା । ସେ ପ୍ରାୟ ୧୦୦ରୁ ଅଧିକ କଥାଚିତ୍ର ଓ ନାଟକରେ ଅଭିନୟ କରିବା ସହିତ ପ୍ରାୟ ୨୧ଟି କଥାଚିତ୍ରରେ ପ୍ରଯୋଜକ ଓ ସହଯୋଗୀ ପ୍ରଯୋଜକ ଭାବରେ କାର୍ଯ୍ୟ କରିଛନ୍ତି ।\n\n##  ଜୀବନୀ\n\n###  ପରିବାର\n\nପ୍ରଦ୍ୟୁମ୍ନ ଲେଙ୍କା ପିତା ଅନ୍ତର୍ଯାମୀ ଲେଙ୍କା ଓ ମାତା ଶୈଳବାଳା ଲେଙ୍କାଙ୍କ ପୁତ୍ର ଭାବେ ୧୯୫୩\n\n###  ପ୍ରାରମ୍ଭିକ ଜୀବନ\n\nପିଲା ଦିନରୁ ସେ ମଞ୍ଚରେ ଅଭିନୟ କରୁଥିଲେ । କଲେଜରେ ଅଭିନୟ କରିବା ସମୟରେ ଗୁରୁ କୃଷ୍ଣଚନ୍ଦ୍ର ସିଂଙ୍କ ସହଯୋଗରେ ପ୍ରଦ୍ୟୁମ୍ନ ୧୯୭୪ ମସିହାରେ ଅନ୍ନପୂର୍ଣ୍ଣା ଥିଏଟରରେ ଅଭିନେତା ଭାବେ ଯୋଗ ଦେଇଥିଲେ ।\n\n###  ଚାକିରି\n\n୧୯୮୧ ମସିହାରେ ପ୍ରଯୋଜକ ଭାବରେ ତାଙ୍କର ପ୍ରଥମ ଚଳଚ୍ଚିତ୍ର ପୂଜା ମୁକ୍ତିଲାଭ କରିଥିଲା । ଅଧ୍ୟାପନାକୁ ପେଷା ଭାବରେ ଗ୍ରହଣ କରିଥିବା ପ୍ରଦ୍ୟୁମ୍ନ , ୨୦୧୩ ମସିହାରେ ଅଳକା ମହାବିଦ୍ୟାଳୟର ଅଧ୍ୟକ୍ଷ ଭାବରେ ଅବସର ଗ୍ରହଣ କରିଛନ୍ତି ।\n\n##  ଫିଲ୍ମରେ ଭୂମିକା\n\n###  ଅଭିନେତା ଭାବରେ\n\n  * ପ୍ରେମ ଏଟିଏମ୍ (୨୦୨୨)\n  * ଚକୋଲେଟ (୨୦୧୧)\n  * ପ୍ରେମ୍ ୱେଡ୍ସ ପ୍ରିୟା (୨୦୧୧)\n  * ତୁ ଥା ମୁଁ ଜାଉଛି ରୁଷି (୨୦୧୦)\n  * ସ୍ୱୟଂସିଦ୍ଧା (୨୦୧୦)\n  * ତୋ ଆଖିରେ ମୁଁ (୨୦୧୦)\n  * ତୁ ଥିଲେ ମୋ ଡର କାହାକୁ (୨୦୧୦)\n  * ସାତ ସୁରେ ବନ୍ଧା ଏ ଜୀବନ (୨୦୦୯)\n  *

In [31]:
body1 = get_body('https://or.wikipedia.org/wiki/%E0%AC%85%E0%AC%B0%E0%AD%8D%E0%AC%9C%E0%AD%81%E0%AC%A8_%E0%AC%AA%E0%AD%81%E0%AC%B0%E0%AC%B8%E0%AD%8D%E0%AC%95%E0%AC%BE%E0%AC%B0')
delete_all_following_elements_by_xpath(body1, adhar_xpath)
delete_elements_by_xpath(body1, xpaths_to_remove)
get_md(body1)


ଆଧାର[ସମ୍ପାଦନା]
Removed element: 

↑ "Pramod Bhagat becomes Odisha's eighth Arjuna awardee". The New Indian Express. Retrieved 2019-08-18.

↑ "क्रिकेटर इंशांत शर्मा और एथलीट दुती चंद समेत 27 खिलाड़ियों को मिलेगा अर्जुन अवॉर्ड". www.zeenews.india.com. ZEE NEWS. Archived from [www.zeenews.india.com/hindi/sports/cricket/ishant-sharma-dutee-chand-among-27-recipients-of-arjuna-award-2020/732795 the original] on 29 August 2020. Retrieved 29 August 2020. {{cite web}}: Check |url= value (help)

↑ "Winner-List-Arjun-Award-Cycling". Ministry of Youth Affairs and Sports, Govt. of India. Retrieved 7 March 2015.


Removed element: ଅଧିକ ତଥ୍ୟ[ସମ୍ପାଦନା]
Removed element: 


ଉଇକିମିଡ଼ିଆ କମନ୍ସରେ  ଅର୍ଜୁନ ପୁରସ୍କାର
ବାବଦରେ ମାଧ୍ୟମ ରହିଛି ।

Removed element: List of Award winners up to 2004
winners for year 2006-2007
winners of 2012 Arjuna Award
http://sports.indiapress.org/arjuna_award.php
http://www.athleteindia.com/2011/07/arjuna-award-winner-2011.html
Removed element: ଏହି ପ୍ରସଙ୍ଗଟି ଅସମ୍ପୂର୍ଣ୍ଣ ଅଟେ । ଆପଣ ଏହାକୁ

'# ଅର୍ଜୁନ ପୁରସ୍କାର\n\n****\n\n  \n\n##  ଅର୍ଜୁନ ପୁରସ୍କାର ବିଜେତା ଓଡ଼ିଆ ଖେଳାଳି\n\nକେନ୍ଦ୍ର ସରକାରଙ୍କ ପ୍ରଚଳିତ ଏହି ସର୍ବୋଚ୍ଚ କ୍ରୀଡ଼ା ସମ୍ମାନର ୫୦+ ବର୍ଷର ଇତିହାସରେ ଓଡ଼ିଶାରୁ ୯ ଜଣ ଅର୍ଜୁନ ପୁରସ୍କାର ବିଜେତା ହୋଇପାରିଛନ୍ତି ।\n\n  1. ପ୍ରମୋଦ ଭଗତ (୨୦୧୯): ପାରା-ବ୍ୟାଡମିଣ୍ଟନ ଖେଳାଳି\n  2. ଦୂତୀ ଚାନ୍ଦ (୨୦୨୦) : ଆଥଲେଟ\n  3. କାଟୁଲୁ ରବି କୁମାର (୨୦୧୧): ଭାରୋତ୍ତଳକ\n  4. ଇଗ୍ନେଶ ତିର୍କି (୨୦୦୯): ହକି ଖେଳାଳି\n  5. ଜ୍ୟୋତି ସୁନିତା କୁଲୁ (୨୦୦୬): ମହିଳା ହକି ଖେଳାଳି\n  6. ଦିଲୀପ ତିର୍କୀ (୨୦୦୨): ହକି ଖେଳାଳି\n  7. ରଚିତା ପଣ୍ଡା (୧୯୯୮): ଆଥଲେଟ\n  8. ବିଜୟ ଶତପଥୀ (୧୯୮୧): ଭାରୋତ୍ତଳକ\n  9. ମିନତି ମହାପାତ୍ର (୧୯୭୮-୭୯): ସାଇକେଲ ଚାଳକ\n\n'

In [32]:
body1 = get_body('https://or.wikipedia.org/wiki/%E0%AC%AA%E0%AC%A6%E0%AD%8D%E0%AC%AE_%E0%AC%AC%E0%AC%BF%E0%AC%AD%E0%AD%82%E0%AC%B7%E0%AC%A3')
delete_all_following_elements_by_xpath(body1, adhar_xpath)
delete_elements_by_xpath(body1, xpaths_to_remove)
get_md(body1)

ଆଧାର[ସମ୍ପାଦନା]
Removed element: 
↑ ୧.୦ ୧.୧ ୧.୨ "Wayback Machine" (PDF). web.archive.org. 2014-05-14. Archived from the original on 2014-05-14. Retrieved 2023-05-05.{{cite web}}:  CS1 maint: bot: original URL status unknown (link)

↑ ୨.୦ ୨.୧ ୨.୨ ୨.୩ "Wayback Machine" (PDF). web.archive.org. 2014-05-18. Archived from the original on 2014-05-18. Retrieved 2023-05-05.{{cite web}}:  CS1 maint: bot: original URL status unknown (link)

↑ Hoiberg, Dale (2000). Students' Britannica India (in ଇଂରାଜୀ). Popular Prakashan. ISBN 978-0-85229-760-5.

↑ https://archive.today/20140517172013/http://timesofindia.indiatimes.com/home/sunday-toi/special-report/The-great-Bharat-Ratna-race/articleshow/2714556.cms

↑ ୫.୦ ୫.୧ Bhattacherje, S. B. (2009-05-01). Encyclopaedia of Indian Events & Dates (in ଇଂରାଜୀ). Sterling Publishers Pvt. Ltd. ISBN 978-81-207-4074-7.

↑ ୬.୦ ୬.୧ Edgar, Thorpe (2011-09). The Pearson General Knowledge Manual 2011 (in ଇଂରାଜୀ). Pearson Education India. ISBN 978-81-317-5640-9. {{cite book

'# ପଦ୍ମ ବିଭୂଷଣ\n\n****\n\n**ପଦ୍ମବିଭୂଷଣ** , ଭାରତ ରତ୍ନ ପରେ ଭାରତୀୟ ଗଣତନ୍ତ୍ରର ଦ୍ୱିତୀୟ ସର୍ବୋଚ୍ଚ ବେସାମରିକ ସମ୍ମାନ ଅଟେ । ୨ ଜାନୁଆରୀ ୧୯୫୪ରେ ସ୍ଥାପିତ ଏହି ପୁରସ୍କାର "ଅସାଧାରଣ ଏବଂ ଉଲ୍ଲେଖନୀୟ ସେବା" ପାଇଁ ପ୍ରଦାନ କରାଯାଏ । ଜାତି, ବୃତ୍ତି, ପଦବୀ କିମ୍ବା ଲିଙ୍ଗ ନିର୍ବିଶେଷରେ ସମସ୍ତ ବ୍ୟକ୍ତି ଏହି ପୁରସ୍କାର ପାଇବାକୁ ଯୋଗ୍ୟ ଅଟନ୍ତି । ତେବେ ଡାକ୍ତର ଓ ବୈଜ୍ଞାନିକଙ୍କ ବ୍ୟତୀତ ରାଷ୍ଟ୍ରାୟତ୍ତ ଉଦ୍ୟୋଗଗୁଡ଼ିକରେ କାର୍ଯ୍ୟ କରୁଥିବା ସରକାରୀ କର୍ମଚାରୀମାନେ ଏହି ପୁରସ୍କାର ପାଇବାକୁ ଯୋଗ୍ୟ ନୁହଁନ୍ତି। ୨୦୨୨ ସୁଦ୍ଧା ୧୯ ଜଣ ମରଣୋତ୍ତର ଓ ୨୧ ଜଣ ଅଣନାଗରିକଙ୍କୁ ମିଶାଇ ୩୨୫ ଜଣଙ୍କୁ ଏହି ପୁରସ୍କାର ପ୍ରଦାନ କରାଯାଇଛି।\n\nପ୍ରତିବର୍ଷ ମେ ୧ରୁ ସେପ୍ଟେମ୍ବର ୧୫ ମଧ୍ୟରେ ଏହି ପୁରସ୍କାର ପାଇଁ ସୁପାରିସ ପଦ୍ମ ପୁରସ୍କାର କମିଟି ନିକଟରେ ଦାଖଲ କରାଯାଏ । ଏହି କମିଟି ଭାରତର ପ୍ରଧାନମନ୍ତ୍ରୀଙ୍କଦ୍ୱାରା ଗଠିତ । ସମସ୍ତ ରାଜ୍ୟ ଓ କେନ୍ଦ୍ରଶାସିତ ଅଞ୍ଚଳ ସରକାର, ଭାରତ ସରକାରଙ୍କ ମନ୍ତ୍ରଣାଳୟ, ଭାରତ ରତ୍ନ ଏବଂ ପୂର୍ବ ପଦ୍ମବିଭୂଷଣ ପୁରସ୍କାର ପ୍ରାପ୍ତ ବ୍ୟକ୍ତି, ଉତ୍କର୍ଷ ପ୍ରତିଷ୍ଠାନ, ମନ୍ତ୍ରୀ, ମୁଖ୍ୟମନ୍ତ୍ରୀ ଓ ରାଜ୍ୟପାଳ ଏବଂ ବ୍ୟକ୍ତିଗତ ବ୍ୟକ୍ତିଙ୍କ ସମେତ ସଂସଦର ସଦସ୍ୟମାନଙ୍କଠାରୁ ଏହି ସୁପାରିସ ଗ୍ରହଣ କରାଯାଇଥାଏ। ପରେ କମିଟି ପରବର୍ତ୍ତୀ ଅନୁମୋଦନ ପାଇଁ ପ୍ରଧାନମନ୍ତ୍ରୀ ଓ ରାଷ୍ଟ୍ରପତିଙ୍କ ନ